In [30]:
"""
In this script, we will
"""

import torch 
import cv2
import numpy as np
import pandas as pd
from sort import sort
from time import time
from collections import defaultdict


from os import listdir

"""
Load the model with a certain confidence threshold (TBD)
"""
model = torch.hub.load('.','custom','best.pt',source='local')

YOLOv5 🚀 163666e80 Python-3.8.10 torch-1.13.0+cu117 CPU

Fusing layers... 
Model summary: 212 layers, 20893344 parameters, 0 gradients
Adding AutoShape... 


In [31]:
model.conf = .27 #Minimum .5 confidence threshold

In [32]:
"""
Initializing values for tracking and video capture:
"""
frameRate = 0
time_limit = 5#Seconds
image_frames = []


#7 days, each index contains new carCount 
countDay = [None]*7
#24 Hours, each index contains new carCount
countHour = [None]*24
#Key:Value = carType: carTypeCount
carCount =defaultdict(int)

frame_count = 0
hourIndex = 0
dayIndex = 0

In [ ]:
"""
USE FOR TESTING IF NEEDED:
"""
path = "data/localization/test/"
images = listdir(path+"images/")
images = [path+"images/"+image for image in images]
videos = listdir(path+"videos/")
videos = [path+"videos/"+video for video in videos]

for video in videos:
    print("Currently processing new video ")
    tracker=sort.Sort()
    #Store tracking info per video
    ids = defaultdict(set)
    carCount = defaultdict(int)
    frame_count = 0
    
    start_time = time()
    vid = cv2.VideoCapture(video)
    ret,frame = vid.read()
    
    frame_count+=1
    #Set number of frames to look at for testing purposes
    while(ret):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        results = model(frame)
        #Retrieving boundingbox data as dataframe 
        #Dataframe Format (xyxy attribute):
        #(xmin,ymin,xmax,ymax,confidence,label_value,label)
        data = results.pandas().xyxy[0]        
        trafficType=data["name"]
        data = data.iloc[:][:5].to_numpy()
        data = data[:,:5].astype('float64')


        #Updated with ids on the camera
        track_res = tracker.update(data)

        if(track_res.size!=0):
            for index,detection in enumerate(track_res[::-1]):
                #If the detection is new, add the ID to the
                #appropriate vehicle/pedestrian set
                if(detection[-1] not in ids[trafficType[index]]):
                    ids[trafficType[index]].add(detection[-1])
                    carCount[trafficType[index]] += 1

        time_elaps = time()-start_time

        if(time_elaps>time_limit):
            frameRate = frame_count/(time_elaps)
            break

        ret, frame = vid.read()
        frame_count +=1

    vid.release()

    print("Frame rate: ",frameRate)
    print(carCount)
    print("Time elapsed:", time()-start_time)

In [37]:
"""
TODO: SETUP PER HOUR & PER DAY SCRIPT

setupFPS count
"""

start_time = time()
#Set number of frames to look at for testing purposes
while(ret):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    results = model(frame)
    #Retrieving boundingbox data as dataframe 
    #Dataframe Format (xyxy attribute):
    #(xmin,ymin,xmax,ymax,confidence,label_value,label)
    data = results.pandas().xyxy[0]        
    trafficType=data["name"]
    data = data.iloc[:][:5].to_numpy()
    data = data[:,:5].astype('float64')


    #Updated with ids on the camera
    track_res = tracker.update(data)

    if(track_res.size!=0):
        for index,detection in enumerate(track_res[::-1]):
            #If the detection is new, add the ID to the
            #appropriate vehicle/pedestrian set
            if(detection[-1] not in ids[trafficType[index]]):
                ids[trafficType[index]].add(detection[-1])
                carCount[trafficType[index]] += 1

    time_elaps = time()-start_time
    if(time_elaps>time_limit):
        frameRate = len(image_frames)/(time_elaps)
        break
    ret, frame = vid.read()

vid.release()
    
print("Frame rate: ",frameRate)
print(carCount)

Frame rate:  0.0
defaultdict(<class 'int'>, {'car': 7, 'motorized_vehicle': 1})
